# T5 - The Kalman filter (KF) -- multivariate
Dealing with vectors and matrices is a lot like plain numbers. But some things get more complicated.
$
% Loading TeX (MathJax)... Please wait
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}}
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}}
%
\newcommand{\trsign}{{\mathsf{T}}}
\newcommand{\tr}{^{\trsign}}
\newcommand{\tn}[1]{#1}
\newcommand{\ceq}[0]{\mathrel{≔}}
\newcommand{\xDim}[0]{D_x}
%
\newcommand{\I}[0]{\mat{I}}
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}}
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$

In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
plt.ion();

## Another time series problem, now multivariate

Recall the AR(1) process from the previous tutorial: $x_{k+1} = \DynMod x_k + q_k$.
- It could result from discretizing [exponential decay](https://en.wikipedia.org/wiki/Exponential_decay):
  $\frac{d x}{d t} = - \beta x \,,$ for some $\beta \geq 0$, and
  adding some white noise, $\frac{d q}{d t}$.
- Discretisation
  - using explicit-Euler produces $\DynMod = (1 - \beta\, \Delta t)$,
  - using implicit-Euler produces $\DynMod = 1/(1 + \beta\, \Delta t)$.
  - such that $x_{k+1}$ is equals the analytic solution requires $\DynMod = e^{- \beta\, \Delta t}$.
  - *PS: note that the 1-st order Taylor expansion of each scheme is the same.*
- Recall that $\{x_k\}$ became a (noisy) constant (horizontal) line when $\DynMod = 1$,
  which makes sense since then $\beta = 0$.
- Similarly, a straight (sloping) line would result from
  $\frac{d^2 x}{d t^2} = 0 \, .$

To make matters more interesting we're now going to consider the $\xDim$-th order model:
  $\displaystyle \frac{d^{\xDim} x}{d t^\xDim} = 0 \, .$
- This can be rewritten as a 1-st order *vector* (i.e. coupled system of) ODE:
  $\frac{d x_i}{d t} = x_{i+1} \, ,$ and $x_{{\xDim}+1} = 0$   
  where the subscript $i$ is now instead the *index* of the state vector element.
- Again we include noise, $\frac{d q_i}{d t}$,
  and damping (exponential decay), $- \beta x_i$, to each component.
- In total, $ \frac{d x_i}{d t} = x_{i+1} - \beta x_i + \frac{d q_i}{d t} \, .$
- Discretizing with time step `dt=1` produces
  $ x_{k+1, i} = x_{k, i+1} + 0.9 x_{k, i} + q_{k, i}\, ,$  
  i.e. $\beta = 0.1$ or $\beta = -\log(0.9)$ depending on which scheme was used.

Thus, $\x_{k+1} = \DynMod \x_k + \q_k$, with $\DynMod$ the matrix specified below.

In [ ]:
xDim = 4 # state (x) length, also model order
M = 0.9*np.eye(xDim) + np.diag(np.ones(xDim-1), 1)
print("M", M, sep="=\n")

K = 100
Q = 0.01**2 * np.diag(1+np.arange(xDim))

# Initial conditions
xa0 = np.zeros(xDim)
Pa0 = 0.1**2 * np.diag(np.arange(xDim))

Note that will generate a $\xDim$-dimensional time series.
But we will only observe the 1st (0th in Python) element/component of the state vector.
We say that the other components are **hidden**.

In [ ]:
H = np.zeros((1, xDim))
H[0, 0] = 1.0
print(f"{H=!s}")

R = .1**2 * np.identity(1)

Simulate synthetic truth (x) and observations (y).

In [ ]:
rnd.seed(4)

In [ ]:
truths, obsrvs = [], []

x = xa0 + np.sqrt(Pa0)@rnd.randn(xDim)
for k in range(K):
    x = M @ x + np.sqrt(Q)@rnd.randn(xDim)
    y = H @ x + np.sqrt(R)@rnd.randn(1)
    truths.append(x)
    obsrvs.append(y)
truths = np.array(truths)
obsrvs = np.array(obsrvs)

for i, x in enumerate(truths.T):
    scaling = (i+1)**4  # for illustration purposes
    plt.plot(scaling*x, label=fr"${scaling}\,x_{i}$")
plt.legend();

In [ ]:
from scipy.linalg import inv
estimates = []
variances = []

xa = xa0
Pa = Pa0
for k in range(K):
    # Forecast step
    xf = M@xa
    Pf = M@Pa@M.T + Q
    # Analysis update step
    Pa = inv( inv(Pf) + H.T@inv(R)@H )
    xa = Pa @ ( inv(Pf)@xf + H.T@inv(R)@obsrvs[k] )
    # Assign
    estimates.append((xf, xa))
    variances.append((Pf, Pa))
    
estimates = np.array(estimates)
variances = np.array(variances)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 6), nrows=xDim, sharex=True)
for i, (truth, estim, ax) in enumerate(zip(truths.T, estimates.T, axs)):
    std = np.sqrt(variances.T[i, i])
    kk = 1 + np.arange(K)
    kk2 = kk.repeat(2)
    ax.plot(kk, truth, c='k')
    ax.plot(kk2, estim.flatten())
    ax.fill_between(kk2, *((estim - std).flatten(),
                           (estim + std).flatten()), alpha=.2)
    ax.set_ylabel(f"$x_{i}$")
    ax.set_xlim([0, K])

## The KF forecast step

The forecast step remains essentially unchanged from the [univariate case](T3%20-%20Bayesian%20inference.ipynb).
The only difference is that $\DynMod$ is now a matrix, as well as the use of the transpose ${}^T$ in the covariance equation:
$\begin{align}
\bb_k
&= \DynMod_{k-1} \hat{\x}_{k-1} \, , \tag{1a} \\\
\B_k
&= \DynMod_{k-1} \bP_{k-1} \DynMod_{k-1}^T + \Q_{k-1} \, . \tag{1b}
\end{align}$

<mark><font size="-1">
We have now derived the Kalman filter, also in the multivariate case. We know how to:
</font></mark>
- Propagate our estimate of $\x$ to the next time step using eqns (1a) and (1b).
- Update our estimate of $\x$ by assimilating the latest observation $\y$, using eqns (5) and (6).

## The KF analysis step

The following exercise derives the analysis step.

#### Exc -- The 'precision' form of the KF
Similarly to [Exc 2.18](T3%20-%20Bayesian%20inference.ipynb#Exc----GG-Bayes),
it may be shown that the prior $p(\x) = \NormDist(\x \mid \bb,\B)$
and likelihood $p(\y|\x) = \NormDist(\y \mid \bH \x,\R)$,
yield the posterior:
\begin{align}
p(\x|\y)
&= \NormDist(\x \mid \hat{\x}, \bP) \tag{4}
\, ,
\end{align}
where the posterior/analysis mean (vector) and covariance (matrix) are given by:
\begin{align}
			\bP &= (\bH\tr \Ri \bH + \Bi)^{-1} \, , \tag{5} \\
			\hat{\x} &= \bP\left[\bH\tr \Ri \y + \Bi \bb\right] \tag{6} \, ,
\end{align}
Prove eqns (4-6).  
*Hint: like the last time, the main part lies in "completing the square" in $\x$.*

In [ ]:
# ws.show_answer('KF precision')

## Implementation & illustration

In [ ]:
def KF(K, xa, Pa, M, H, Q, R, obsrvs):
    """Kalman filter. PS: (xa, Pa) should be input with *initial* values."""
    ############################
    # TEMPORARY IMPLEMENTATION #
    ############################
    estimates = np.zeros((K, 2))
    variances = np.zeros((K, 2))
    for k in range(K):
        # Forecast step
        xf = xa
        Pf = Pa
        # Analysis update step
        Pa = R / H**2
        xa = obsrvs[k] / H
        # Assign
        estimates[k] = xf, xa
        variances[k] = Pf, Pa
    return estimates, variances

## Woodbury and the Kalman gain
The KF formulae, as specified above, can be pretty expensive...

#### Exc (optional) -- flops and MBs
Suppose $\x$ is $\xDim$-dimensional and has a covariance matrix $\B$.
 * (a). What's the size of $\B$?
 * (b). How many "flops" (approximately, i.e. to leading order) are required  
 to compute the "precision form" of the KF update equation, eqn (5) ?
 * (c). How much memory (bytes) is required to hold its covariance matrix $\B$ ?
 * (d). How many megabytes (MB) is this if $\xDim$ is a million?

In [ ]:
# ws.show_answer('nD-covars are big)

This is one of the principal reasons why basic extended KF is infeasible for DA.  
The following derives another, often more practical, form of the KF analysis update.

#### Exc -- The "Woodbury" matrix inversion identity
The following is known as the Sherman-Morrison-Woodbury lemma/identity,
$$\begin{align}
    \bP = \left( \B^{-1} + \V\tr \R^{-1} \U \right)^{-1}
    =
    \B - \B \V\tr \left( \R + \U \B \V\tr \right)^{-1} \U \B \, ,
    \tag{W}
\end{align}$$
which holds for any (suitably shaped matrices)
$\B$, $\R$, $\V,\U$ *such that the above exists*.

Prove the identity. Hint: don't derive it, just prove it!

In [ ]:
# ws.show_answer('Woodbury')

#### Exc (optional) -- Matrix shape compatibility
- Show that $\B$ and $\R$ must be square.
- Show that $\U$ and $\V$ are not necessarily square, but must have the same dimensions.
- Show that $\B$ and $\R$ are not necessarily of equal size.


Exc 7 makes it clear that the Woodbury identity may be used to compute $\bP$ by inverting matrices of the size of $\R$ rather than the size of $\B$.
Of course, if $\R$ is bigger than $\B$, then the identity is useful the other way around.

#### Exc (optional) -- Corollary 1
Prove that, for any symmetric, positive-definite (SPD) matrices $\R$ and $\B$, and any matrix $\bH$,
$$\begin{align}
 	\left(\bH\tr \R^{-1} \bH + \B^{-1}\right)^{-1}
    &=
    \B - \B \bH\tr \left( \R + \bH \B \bH\tr \right)^{-1} \bH \B \tag{C1}
    \, .
\end{align}$$
*Hint: consider the properties of [SPD](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix#Properties) matrices.*

In [ ]:
# ws.show_answer('Woodbury C1')

#### Exc (optional) -- Corollary 2
Prove that, for the same matrices as for Corollary C1,
$$\begin{align}
	\left(\bH\tr \R^{-1} \bH + \B^{-1}\right)^{-1}\bH\tr \R^{-1}
    &= \B \bH\tr \left( \R + \bH \B \bH\tr \right)^{-1}
    \tag{C2}
    \, .
\end{align}$$

In [ ]:
# ws.show_answer('Woodbury C2')

#### Exc -- The "Gain" form of the KF
Now, let's go back to the KF, eqns (5) and (6). Since $\B$ and $\R$ are covariance matrices, they are symmetric-positive. In addition, we will assume that they are full-rank, making them SPD and invertible.  

Define the Kalman gain by:
 $$\begin{align}
    \K &= \B \bH\tr \big(\bH \B \bH\tr + \R\big)^{-1} \, . \tag{K1}
\end{align}$$
 * (a) Apply (C1) to eqn (5) to obtain the Kalman gain form of analysis/posterior covariance matrix:
$$\begin{align}
    \bP &= [\I_{\xDim} - \K \bH]\B \, . \tag{8}
\end{align}$$

* (b) Apply (C2)  to (5) to obtain the identity
$$\begin{align}
    \K &= \bP \bH\tr \R^{-1}  \, . \tag{K2}
\end{align}$$

* (c) Show that $\bP \Bi = [\I_{\xDim} - \K \bH]$.
* (d) Use (b) and (c) to obtain the Kalman gain form of analysis/posterior covariance
$$\begin{align}
     \hat{\x} &= \bb + \K\left[\y - \bH \bb\right] \, . \tag{9}
\end{align}$$
Together, eqns (8) and (9) define the Kalman gain form of the KF update.
Note that the inversion (eqn 7) involved is of the size of $\R$, while in eqn (5) it is of the size of $\B$.

#### Exc -- KF with gain
Implement the Kalman gain form in place of the precision form of the KF.

## Summary
We have derived two forms of the multivariate KF analysis update step: the "precision matrix" form, and the "Kalman gain" form. The latter is especially practical when the number of observations is smaller than the length of the state vector. Still, the best is yet to come: the ability to handle very large and chaotic systems
(which are more fun than stochastically driven signals such as above).

### Next: [T6 - Spatial statistics ("geostatistics") & Kriging](T6%20-%20Geostatistics%20%26%20Kriging.ipynb)